## Control Entropy

Specs:
- Low Back IMU 
- Sampled at 1125 hz
- All runs 5 minutes 

In [ ]:
# Import custom functions ---
import functions.file_import_gui as gui
import functions.data_prep as prep
import functions.custom_plots as plots
import functions.low_back_measures as back

# For saving files
import os

# For entropy
import nolds

# For dataframes ---
import pandas as pd
import numpy as np

# For plotting ---
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Bring in IMU data files ---

# Subject to process
sub_id = 'run014'

# lowg ---
# set directory
initialdir = f"data/five_min_runs/{sub_id}/lowg_1125hz/back"
# bring in csv files with data
dfs_lowg_lowg_lowg, keys_list = gui.read_csv_files_gui(initialdir)

Prep

In [ ]:
# Data prep ---

# lowg ---
# crop data for 5 mins (removes extra rows at the beginning so that there are exactly 5 mins of data)
prep.crop_df_five_mins(dfs_lowg_lowg_lowg, sample_freq = 1125)
# calculate and add resultant column
prep.add_resultant_column(dfs_lowg_lowg_lowg, column_x = 'ax_m/s/s', column_y = 'ay_m/s/s', column_z = 'az_m/s/s', name_of_res_column = 'res_m/s/s')
# convert accel columns to gs
prep.accel_to_gs_columns(dfs_lowg_lowg_lowg)
# shift time scale to start at 0
prep.shift_time_s_to_zero(dfs_lowg_lowg_lowg)

Calculation

In [ ]:
# Control Entropy ---
# Method with overlapping window

# set window size and overlap
window_size = 750
overlap = 375  # overlapping window of 50%

# initialize dictionary to store sample entropy values for each dataframe
sample_entropies = {}

# loop through dataframes in dictionary
for key, df in dfs_lowg.items():
    # get signal column from dataframe
    signal = df['res_m/s/s'].values

    # initialize list to store sample entropy values for each window
    sample_entropy_values = []

    # calculate sample entropy for each window
    # moves over 375 each time and then ends its window at whearever it started plus 750
    for i in range(0, len(signal) - window_size, overlap):
        window = signal[i:i+window_size]
        sample_entropy = nolds.sampen(window, emb_dim=2, tolerance=0.15)
        sample_entropy_values.append(sample_entropy)

    # calculate the average sample entropy value for the entire signal
    average_sample_entropy = np.mean(sample_entropy_values)

    # store the average sample entropy value for this dataframe in the dictionary
    sample_entropies[key] = average_sample_entropy

# convert the dictionary to a dataframe for easier manipulation and analysis
sample_entropies_df = pd.DataFrame.from_dict(
    sample_entropies, orient='index', columns=['sample_entropy'])

Export

In [ ]:
# Create table to export ---

# create the variable_export_df dataframe
variable_export_df = pd.DataFrame(
    columns=['sub_id', 'run_type', 'sensor', 'variable', 'value'])

# loop through each row in the sample_entropies_df dataframe
for index, row in sample_entropies_df.iterrows():

    # extract the sub_id
    sub_id = index.split('_')[0]

    # extract the run_type
    run_type_parts = index.split('_')[1:-2]
    run_type = '_'.join(
        [part for part in run_type_parts if not part.isdigit()])

    # extract the sensor
    sensor = None
    for word in index.split('_'):
        if len(word) == 5 and word.isdigit():
            sensor = word
            break

    # add a new row to the variable_export_df dataframe
    variable_export_df = variable_export_df.append({'sub_id': sub_id,
                                                    'run_type': run_type,
                                                    'sensor': sensor,
                                                    'variable': 'control entropy',
                                                    'value': row['sample_entropy']},
                                                   ignore_index=True)

In [ ]:
# Append rows to Excel Table ---

# define the file path
file_path = "data/processed_variables/imu_training_load_variables.xlsx"

# load existing excel file (if it exists) or create a new one (if it does not exist)
try:
    with pd.ExcelFile(file_path) as xlsx:
        df = pd.read_excel(xlsx, sheet_name='variables')
except FileNotFoundError:
    df = pd.DataFrame()

# append variable_export_df to the existing data in the excel file
df = df.append(variable_export_df, ignore_index=True)

# write the updated dataframe to the excel file
with pd.ExcelWriter(file_path) as writer:
    df.to_excel(writer, sheet_name='variables', index=False)

# Print a message to indicate that the file has been saved
print(f'{sub_id} saved successfully')